# Graph neat examples in Raku

<span style="font-size: 16pt; font-style: italic; font-weight: bold">Set 4</span>

Anton Antonov   
[RakuForPrediction at WordPress](https://rakuforprediction.wordpress.com)   
[RakuForPrediction-book at GitHub](https://github.com/antononcube/RakuForPrediction-book)      
November 2024  
December 2024

-----

## Introduction

**What is a neat example?** : Concise or straightforward code that produces compelling visual or textual outputs.

**Maybe:** We know *neat* when we see it?

The neat examples:

- Showcase Raku programming.
- Use functionalities of different Raku modules.
- Give interesting perspectives on what is computationally possible.

Showcased:
- All computational graph features discussed here are provided by ["Graph"](https://raku.land/zef:antononcube/Graph).   
- Graph plotting with:
    - `js-d3-graph-plot`, provided by ["JavaScript::D3"](https://raku.land/zef:antononcube/JavaScript::D3).
    -  `Graph.dot`, that makes SVG images via [Graphviz](https://graphviz.org).

------

## Setup

Here are loaded the packages used in the rest of notebook:

In [43]:
use Graph;

use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::HexagonalGrid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Nested;
use Graph::Path;
use Graph::Petersen;
use Graph::Star;
use Graph::TriangularGrid;
use Graph::Wheel;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;
use Data::Generators;
use Data::TypeSystem;
use Data::Translators;
use Data::Geographics;

use Math::DistanceFunctions;
use Math::Nearest;
use Math::SparseMatrix;
use Text::Levenshtein::Damerau;

use Hash::Merge;
use FunctionalParsers;
use FunctionalParsers::EBNF;
use EBNF::Grammar;
use Graphviz::DOT::Grammar;

use JavaScript::D3;
use WWW::MermaidInk;

use paths;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [44]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [45]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [46]:
my $background = '#1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $stroke-color = 'SlateGray';
my $tick-labels-color = 'Silver';
my $tick-labels-font-family = 'Helvetica';
my $tick-labels-font-size = 10;
my $title-color = 'Ivory';
my $tooltip-background-color = 'none';
my $tooltip-color = 'LightBlue';
my $vertex-size = 6;
my $engine = 'neato';
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

my %opts = :$background, :$title-color, :$edge-thickness, :$vertex-size;

{background => #1F1F1F, edge-thickness => 3, title-color => Ivory, vertex-size => 6}

------

## Array of SVG plots

- Elegant iteration using graph classes *partial names* 
- It is possible to make a row of SVG plots (made, say, by Graphviz DOT) using the HTML output magic cell: 

In [47]:
#%html
my %opts = node-width => 0.2, node-fill-color => 'SlateBlue', node-height => 0.2, node-font-size => 6, engine => 'neato', size => (3, 3), :svg;

<Cycle Star Wheel>
    .map({ Graph::{$_}.new(8).dot(|%opts) })
    .join("\n\n")


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--6 -->
 
 0--6 
 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 0--4 -->
 
 0--4 
 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 0--8 -->
 
 0--8 
 
 
<!-- 0--2 -->
 
 0--2 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 0--3 -->
 
 0--3 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--8 -->
 
 0--8 
 
 
<!-- 0--4 -->
 
 0--4 
 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 0--2 -->
 
 0--2 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 0--6 -->
 
 0--6 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 1--8 -->
 
 1--8 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 6--7 -->
 
 6--7

Here is row of [circulant graphs](https://en.wikipedia.org/wiki/Circulant_graph):

In [48]:
#% html
my @gs = [3, 5, 10, 13].map({ Graph::Circulant.new(17, [1, $_]) });
@gs.map({ $_.dot(|%opts) }).join("\n\n")

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 13 -->
 
 13 
 
 13 
 
<!-- 14 -->
 
 14 
 
 14 
 
<!-- 13--14 -->
 
 13--14 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 13--16 -->
 
 13--16 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 0--14 -->
 
 0--14 
 
 
<!-- 0--16 -->
 
 0--16 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 5--8 -->
 
 5--8 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- 11--14 -->
 
 11--14 
 
 
<!-- 11--8 -->
 
 11--8 
 
 
<!-- 12 -->
 
 12 
 
 12 
 
<!-- 11--12 -->
 
 11--12 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 3--6 -->
 
 3--6 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 14--15 -->
 
 14--15 
 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 8--9 -->
 
 8--9 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 16--2 -->
 
 16--2 
 
 
<!-- 15--16 -->
 
 15--16 
 
 
<!-- 1--15 -->
 
 1--15 
 
 
<!-- 1--4 -->
 
 1--4 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 12--13 -->
 
 12--13 
 
 
<!-- 12--15 -->
 
 12--15 
 
 
<!-- 12--9 -->
 
 12--9 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 4--7 -->
 
 4--7 
 
 
<!-- 2--5 -->
 
 2--5 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 10--13 -->
 
 10--13 
 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 10--7 -->
 
 10--7 
 
 
<!-- 10--9 -->
 
 10--9 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 6--9 -->
 
 6--9 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 9 -->
 
 9 
 
 9 
 
<!-- 8--9 -->
 
 8--9 
 
 
<!-- 12 -->
 
 12 
 
 12 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 12--7 -->
 
 12--7 
 
 
<!-- 13 -->
 
 13 
 
 13 
 
<!-- 12--13 -->
 
 12--13 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 2--7 -->
 
 2--7 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 11 -->
 
 11 
 
 11 
 
<!-- 11--12 -->
 
 11--12 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 11--6 -->
 
 11--6 
 
 
<!-- 16 -->
 
 16 
 
 16 
 
<!-- 11--16 -->
 
 11--16 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 14 -->
 
 14 
 
 14 
 
<!-- 14--2 -->
 
 14--2 
 
 
<!-- 14--9 -->
 
 14--9 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 14--15 -->
 
 14--15 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 4--9 -->
 
 4--9 
 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 10 -->
 
 10 
 
 10 
 
<!-- 10--11 -->
 
 10--11 
 
 
<!-- 10--5 -->
 
 10--5 
 
 
<!-- 10--9 -->
 
 10--9 
 
 
<!-- 10--15 -->
 
 10--15 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--12 -->
 
 0--12 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 0--16 -->
 
 0--16 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 16--4 -->
 
 16--4 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 1--6 -->
 
 1--6 
 
 
<!-- 1--13 -->
 
 1--13 
 
 
<!-- 15--16 -->
 
 15--16 
 
 
<!-- 15--3 -->
 
 15--3 
 
 
<!-- 3--8 -->
 
 3--8 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 13--8 -->
 
 13--8 
 
 
<!-- 13--14 -->
 
 13--14 
 
 
 
 


<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 8 -->
 
 8 
 
 8 
 
<!-- 7--8 -->
 
 7--8 
 
 
<!-- 14 -->
 
 14 
 
 14 
 
<!-- 14--7 -->
 
 14--7 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 14--4 -->
 
 14--4 
 
 
<!-- 15 -->
 
 15 
 
 15 
 
<!-- 14--15 -->
 
 14--15 
 
 
<!-- 13 -->
 
 13 
 
 13 
 
<!-- 13--14 -->
 
 13--14 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 13--6 -->
 
 13--6 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 13--3 -->
 
 13--3 
 
 
<!-- 2 -

-----

## Sparse matrix connection

There is a natural representation of graphs into sparse matrices.

(The implementation of the "Math::SparseMatrix" package became high priority because of "Graph".)

In [49]:
#% js
@gs.map({
    my $m = Math::SparseMatrix.new(edge-dataset => $_.edges(:dataset), row-names => $_.vertex-list.sort(*.Int));
    $m.Array ==> js-d3-matrix-plot(width => 300, margins => 15, :!tooltip, :$tick-labels-font-size, :$tick-labels-color, color-palette => 'Inferno')
}).join("\n")

-----

## Another chessboard making

Chessboards can be via circulant graphs using their adjacency matrices:

In [50]:
#% html
#my $g = Graph::Circulant.new(6, [3, 5]);
my $g = Graph::Circulant.new(8, [3, 5, 7]);
$g.dot(:$engine):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 4 -->
 
 4 
 
 4 
 
<!-- 7 -->
 
 7 
 
 7 
 
<!-- 4--7 -->
 
 4--7 
 
 
<!-- 5 -->
 
 5 
 
 5 
 
<!-- 4--5 -->
 
 4--5 
 
 
<!-- 0 -->
 
 0 
 
 0 
 
<!-- 0--7 -->
 
 0--7 
 
 
<!-- 3 -->
 
 3 
 
 3 
 
<!-- 0--3 -->
 
 0--3 
 
 
<!-- 0--5 -->
 
 0--5 
 
 
<!-- 1 -->
 
 1 
 
 1 
 
<!-- 0--1 -->
 
 0--1 
 
 
<!-- 3--4 -->
 
 3--4 
 
 
<!-- 6 -->
 
 6 
 
 6 
 
<!-- 3--6 -->
 
 3--6 
 
 
<!-- 2 -->
 
 2 
 
 2 
 
<!-- 2--7 -->
 
 2--7 
 
 
<!-- 2--3 -->
 
 2--3 
 
 
<!-- 2--5 -->
 
 2--5 
 
 
<!-- 5--6 -->
 
 5--6 
 
 
<!-- 6--7 -->
 
 6--7 
 
 
<!-- 1--4 -->
 
 1--4 
 
 
<!-- 1--2 -->
 
 1--2 
 
 
<!-- 1--6 -->
 
 1--6

In [51]:
#% js
 my $m = Math::SparseMatrix.new(edge-dataset => $g.edges(:dataset), row-names => $g.vertex-list.sort(*.Int));
 say $m;
 $m.Array ==> js-d3-matrix-plot(width => 400, margins => 15, :$tick-labels-font-size, :$tick-labels-color, color-palette => 'YlOrBr')

Math::SparseMatrix(:specified-elements(32), :dimensions((8, 8)), :density(0.5))


------

## Grid with holes and highlights

In this section we show how to subtract random neighborhoods from a grid graph and plot the result.

In [52]:
my $g = Graph::TriangularGrid.new(30, 30);
#my $g = Graph::Grid.new(25, 50);

#my $vset = $g.vertex-list (-) $g.neighborhood-graph($g.vertex-list.pick(60), d => 1).vertex-list;
#my $g2 = $g.subgraph($vset.keys)
my $g2 = $g.difference( $g.neighborhood-graph($g.vertex-list.pick(60), d => 1) );

Graph(vertexes => 976, edges => 2124, directed => False)

In [53]:
my $g3 = $g2.neighborhood-graph($g2.vertex-list.pick(60), d => 2);
my $nc = $g3.weakly-connected-components.elems;
my @comps = $g3.weakly-connected-components;
my @gs = @comps.map({ $g3.subgraph($_) });
my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
$nc

10

In [54]:
@highlight.pick(12)

(404 19 => 23 54 => 69 948 => 957 916 => 924 930 => 937 436 => 467 617 => 663 31 => 43 796 836 => 847 148 => 161)

In [55]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        vertex-coordinates => $g2.vertex-coordinates,
        highlight => {Ivory => @highlight},
        :$background, 
        title-color => 'none', 
        width => 800, 
        height => 500,
        edge-thickness => 3,
        edge-color => 'Gray',
        vertex-size => 0,
        vertex-color => 'Gray',
        force => {charge => {strength => -30, iterations => 2}, y => {strength => 0.2}, collision => {radius => 1, iterations => 1}, link => {minDistance => 1}}
    )

Graphviz DOT rendering:

In [57]:
#%html
$g2.dot( 
    highlight => {Ivory => @highlight},
    :!node-labels, 
    node-width => 0,
    node-fill-color => 'Gray', 
    node-shape => 'point',
    graph-size => 8,
    edge-color => 'Gray',
    edge-width => 28,
    engine => 'neato', 
):svg

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 720 -->
 
 720 
 
 
<!-- 309 -->
 
 309 
 
 
<!-- 324 -->
 
 324 
 
 
<!-- 309--324 -->
 
 309--324 
 
 
<!-- 340 -->
 
 340 
 
 
<!-- 309--340 -->
 
 309--340 
 
 
<!-- 325 -->
 
 325 
 
 
<!-- 309--325 -->
 
 309--325 
 
 
<!-- 847 -->
 
 847 
 
 
<!-- 868 -->
 
 868 
 
 
<!-- 847--868 -->
 
 847--868 
 
 
<!-- 857 -->
 
 857 
 
 
<!-- 847--857 -->
 
 847--857 
 
 
<!-- 858 -->
 
 858 
 
 
<!-- 847--858 -->
 
 847--858 
 
 
<!-- 306 -->
 
 306 
 
 
<!-- 337 -->
 
 337 
 
 
<!-- 306--337 -->
 
 306--337 
 
 
<!-- 321 -->
 
 321 
 
 
<!-- 306--321 -->
 
 306--321 
 
 
<!-- 322 -->
 
 322 
 
 
<!-- 306--322 -->
 
 306--322 
 
 
<!-- 597 -->
 
 597 
 
 
<!-- 628 -->
 
 628 
 
 
<!-- 597--628 -->
 
 597--628 
 
 
<!-- 612 -->
 
 612 
 
 
<!-- 597--612 -->
 
 597--612 
 
 
<!-- 613 -->
 
 613 
 
 
<!-- 597--613 -->
 
 597--613 
 
 
<!-- 591 -->
 
 591 
 
 
<!-- 728 -->
 
 728 
 
 
<!-- 744 -->
 
 744 
 
 
<!-- 728--744 -->
 
 728--744 
 
 
<!-- 758 -->
 
 758 
 
 
<!-- 728--758 -->
 
 728--758 
 
 
<!-- 743 -->
 
 743 
 
 
<!-- 728--743 -->
 
 728--743 
 
 
<!-- 811 -->
 
 811 
 
 
<!-- 823 -->
 
 823 
 
 
<!-- 811--823 -->
 
 811--823 
 
 
<!-- 835 -->
 
 835 
 
 
<!-- 811--835 -->
 
 811--835 
 
 
<!-- 824 -->
 
 824 
 
 
<!-- 811--824 -->
 
 811--824 
 
 
<!-- 57 -->
 
 57 
 
 
<!-- 65 -->
 
 65 
 
 
<!-- 57--65 -->
 
 57--65 
 
 
<!-- 637 -->
 
 637 
 
 
<!-- 667 -->
 
 667 
 
 
<!-- 637--667 -->
 
 637--667 
 
 
<!-- 120 -->
 
 120 
 
 
<!-- 130 -->
 
 130 
 
 
<!-- 120--130 -->
 
 120--130 
 
 
<!-- 142 -->
 
 142 
 
 
<!-- 120--142 -->
 
 120--142 
 
 
<!-- 131 -->
 
 131 
 
 
<!-- 120--131 -->
 
 120--131 
 
 
<!-- 830 -->
 
 830 
 
 
<!-- 842 -->
 
 842 
 
 
<!-- 830--842 -->
 
 830--842 
 
 
<!-- 841 -->
 
 841 
 
 
<!-- 830--841 -->
 
 830--841 
 
 
<!-- 853 -->
 
 853 
 
 
<!-- 830--853 -->
 
 830--853 
 
 
<!-- 68 -->
 
 68 
 
 
<!-- 85 -->
 
 85 
 
 
<!-- 68--85 -->
 
 68--85 
 
 
<!-- 76 -->
 
 76 
 
 
<!-- 68--76 -->
 
 68--76 
 
 
<!-- 77 -->
 
 77 
 
 
<!-- 68--77 -->
 
 68--77 
 
 
<!-- 642 -->
 
 642 
 
 
<!-- 673 -->
 
 673 
 
 
<!-- 642--673 -->
 
 642--673 
 
 
<!-- 672 -->
 
 672 
 
 
<!-- 642--672 -->
 
 642--672 
 
 
<!-- 688 -->
 
 688 
 
 
<!-- 642--688 -->
 
 642--688 
 
 
<!-- 180 -->
 
 180 
 
 
<!-- 193 -->
 
 193 
 
 
<!-- 180--193 -->
 
 180--193 
 
 
<!-- 207 -->
 
 207 
 
 
<!-- 180--207 -->
 
 180--207 
 
 
<!-- 194 -->
 
 194 
 
 
<!-- 180--194 -->
 
 180--194 
 
 
<!-- 269 -->
 
 269 
 
 
<!-- 285 -->
 
 285 
 
 
<!-- 269--285 -->
 
 269--285 
 
 
<!-- 284 -->
 
 284 
 
 
<!-- 269--284 -->
 
 269--284 
 
 
<!-- 300 -->
 
 300 
 
 
<!-- 269--300 -->
 
 269--300 
 
 
<!-- 281 -->
 
 281 
 
 
<!-- 646 -->
 
 646 
 
 
<!-- 692 -->
 
 692 
 
 
<!-- 646--692 -->
 
 646--692 
 
 
<!-- 677 -->
 
 677 
 
 
<!-- 646--677 -->
 
 646--677 
 
 
<!-- 170 -->
 
 170 
 
 
<!-- 197 -->
 
 197 
 
 
<!-- 170--197 -->
 
 170--197 
 
 
<!-- 183 -->
 
 183 
 
 
<!-- 170--183 -->
 
 170--183 
 
 
<!-- 184 -->
 
 184 
 
 
<!-- 170--184 -->
 
 170--184 
 
 
<!-- 168 -->
 
 168 
 
 
<!-- 168--180 -->
 
 168--180 
 
 
<!-- 181 -->
 
 181 
 
 
<!-- 168--181 -->
 
 168--181 
 
 
<!-- 168--194 -->
 
 168--194 
 
 
<!-- 268 -->
 
 268 
 
 
<!-- 268--284 -->
 
 268--284 
 
 
<!-- 299 -->
 
 299 
 
 
<!-- 268--299 -->
 
 268--299 
 
 
<!-- 283 -->
 
 283 
 
 
<!-- 268--283 -->
 
 268--283 
 
 
<!-- 711 -->
 
 711 
 
 
<!-- 742 -->
 
 742 
 
 
<!-- 711--742 -->
 
 711--742 
 
 
<!-- 726 -->
 
 726 
 
 
<!-- 711--726 -->
 
 711--726 
 
 
<!-- 727 -->
 
 727 
 
 
<!-- 711--727 -->
 
 711--727 
 
 
<!-- 732 -->
 
 732 
 
 
<!-- 748 -->
 
 748 
 
 
<!-- 732--748 -->
 
 732--748 
 
 
<!-- 762 -->
 
 762 
 
 
<!-- 732--762 -->
 
 732--762 
 
 
<!-- 747 -->
 
 747 
 
 
<!-- 732--747 -->
 
 

-----

## Subgraph using vertex coordinates

Same as above but with ***Hexagonal Grid***:

In [34]:
my $g = Graph::HexagonalGrid.new(20, 20);
my $g2 = $g.difference( $g.neighborhood-graph($g.vertex-list.pick(60), d => 1) );

Graph(vertexes => 882, edges => 1100, directed => False)

In [35]:
#% html
$g.dot(:$engine, edge-thickness => 20, vertex-width => 0.9, :!vertex-labels, size => 8):svg;

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 381 -->
 
 381 
 
 
<!-- 402 -->
 
 402 
 
 
<!-- 381--402 -->
 
 381--402 
 
 
<!-- 213 -->
 
 213 
 
 
<!-- 234 -->
 
 234 
 
 
<!-- 213--234 -->
 
 213--234 
 
 
<!-- 290 -->
 
 290 
 
 
<!-- 311 -->
 
 311 
 
 
<!-- 290--311 -->
 
 290--311 
 
 
<!-- 291 -->
 
 291 
 
 
<!-- 290--291 -->
 
 290--291 
 
 
<!-- 510 -->
 
 510 
 
 
<!-- 511 -->
 
 511 
 
 
<!-- 510--511 -->
 
 510--511 
 
 
<!-- 531 -->
 
 531 
 
 
<!-- 510--531 -->
 
 510--531 
 
 
<!-- 136 -->
 
 136 
 
 
<!-- 137 -->
 
 137 
 
 
<!-- 136--137 -->
 
 136--137 
 
 
<!-- 153 -->
 
 153 
 
 
<!-- 136--153 -->
 
 136--153 
 
 
<!-- 203 -->
 
 203 
 
 
<!-- 223 -->
 
 223 
 
 
<!-- 203--223 -->
 
 203--223 
 
 
<!-- 204 -->
 
 204 
 
 
<!-- 203--204 -->
 
 203--204 
 
 
<!-- 586 -->
 
 586 
 
 
<!-- 607 -->
 
 607 
 
 
<!-- 586--607 -->
 
 586--607 
 
 
<!-- 64 -->
 
 64 
 
 
<!-- 75 -->
 
 75 
 
 
<!-- 64--75 -->
 
 64--75 
 
 
<!-- 637 -->
 
 637 
 
 
<!-- 658 -->
 
 658 
 
 
<!-- 637--658 -->
 
 637--658 
 
 
<!-- 159 -->
 
 159 
 
 
<!-- 177 -->
 
 177 
 
 
<!-- 159--177 -->
 
 159--177 
 
 
<!-- 532 -->
 
 532 
 
 
<!-- 533 -->
 
 533 
 
 
<!-- 532--533 -->
 
 532--533 
 
 
<!-- 553 -->
 
 553 
 
 
<!-- 532--553 -->
 
 532--553 
 
 
<!-- 545 -->
 
 545 
 
 
<!-- 566 -->
 
 566 
 
 
<!-- 545--566 -->
 
 545--566 
 
 
<!-- 368 -->
 
 368 
 
 
<!-- 389 -->
 
 389 
 
 
<!-- 368--389 -->
 
 368--389 
 
 
<!-- 369 -->
 
 369 
 
 
<!-- 368--369 -->
 
 368--369 
 
 
<!-- 817 -->
 
 817 
 
 
<!-- 818 -->
 
 818 
 
 
<!-- 817--818 -->
 
 817--818 
 
 
<!-- 827 -->
 
 827 
 
 
<!-- 817--827 -->
 
 817--827 
 
 
<!-- 652 -->
 
 652 
 
 
<!-- 672 -->
 
 672 
 
 
<!-- 652--672 -->
 
 652--672 
 
 
<!-- 653 -->
 
 653 
 
 
<!-- 652--653 -->
 
 652--653 
 
 
<!-- 219 -->
 
 219 
 
 
<!-- 240 -->
 
 240 
 
 
<!-- 219--240 -->
 
 219--240 
 
 
<!-- 720 -->
 
 720 
 
 
<!-- 721 -->
 
 721 
 
 
<!-- 720--721 -->
 
 720--721 
 
 
<!-- 737 -->
 
 737 
 
 
<!-- 720--737 -->
 
 720--737 
 
 
<!-- 874 -->
 
 874 
 
 
<!-- 877 -->
 
 877 
 
 
<!-- 874--877 -->
 
 874--877 
 
 
<!-- 630 -->
 
 630 
 
 
<!-- 651 -->
 
 651 
 
 
<!-- 630--651 -->
 
 630--651 
 
 
<!-- 631 -->
 
 631 
 
 
<!-- 630--631 -->
 
 630--631 
 
 
<!-- 692 -->
 
 692 
 
 
<!-- 710 -->
 
 710 
 
 
<!-- 692--710 -->
 
 692--710 
 
 
<!-- 432 -->
 
 432 
 
 
<!-- 433 -->
 
 433 
 
 
<!-- 432--433 -->
 
 432--433 
 
 
<!-- 453 -->
 
 453 
 
 
<!-- 432--453 -->
 
 432--453 
 
 
<!-- 816 -->
 
 816 
 
 
<!-- 826 -->
 
 826 
 
 
<!-- 816--826 -->
 
 816--826 
 
 
<!-- 171 -->
 
 171 
 
 
<!-- 190 -->
 
 190 
 
 
<!-- 171--190 -->
 
 171--190 
 
 
<!-- 172 -->
 
 172 
 
 
<!-- 171--172 -->
 
 171--172 
 
 
<!-- 867 -->
 
 867 
 
 
<!-- 875 -->
 
 875 
 
 
<!-- 867--875 -->
 
 867--875 
 
 
<!-- 751 -->
 
 751 
 
 
<!-- 752 -->
 
 752 
 
 
<!-- 751--752 -->
 
 751--752 
 
 
<!-- 766 -->
 
 766 
 
 
<!-- 751--766 -->
 
 751--766 
 
 
<!-- 584 -->
 
 584 
 
 
<!-- 605 -->
 
 605 
 
 
<!-- 584--605 -->
 
 584--605 
 
 
<!-- 585 -->
 
 585 
 
 
<!-- 584--585 -->
 
 584--585 
 
 
<!-- 9 -->
 
 9 
 
 
<!-- 175 -->
 
 175 
 
 
<!-- 194 -->
 
 194 
 
 
<!-- 175--194 -->
 
 175--194 
 
 
<!-- 176 -->
 
 176 
 
 
<!-- 175--176 -->
 
 175--176 
 
 
<!-- 708 -->
 
 708 
 
 
<!-- 726 -->
 
 726 
 
 
<!-- 708--726 -->
 
 708--726 
 
 
<!-- 115 -->
 
 115 
 
 
<!-- 116 -->
 
 116 
 
 
<!-- 115--116 -->
 
 115--116 
 
 
<!-- 130 -->
 
 130 
 
 
<!-- 115--130 -->
 
 115--130 
 
 
<!-- 161 -->
 
 161 
 
 
<!-- 179 -->
 
 179 
 
 
<!-- 161--179 -->
 
 161--179 
 
 
<!-- 169 -->
 
 169 
 
 
<!-- 187 -->
 
 187 
 
 
<!-- 169--187 -->
 
 169--187 
 
 
<!-- 741 -->
 
 741 
 
 
<!-- 757 -->
 
 757 
 
 
<!-- 741--757 -->
 
 741--757 
 
 
<!-- 761 -->
 
 761 
 
 
<!-- 762 -->
 
 762 
 
 
<!-- 761--762 -

Summary of the coordinates:

In [36]:
sink records-summary($g2.vertex-coordinates.values, field-names => ['0', '1'])

+-----------------------------+----------------------------+
| 0                           | 1                          |
+-----------------------------+----------------------------+
| Min    => -5.196152422707   | Min    => -4               |
| 1st-Qu => 27.712812921102   | 1st-Qu => 11               |
| Mean   => 46.00342655114355 | Mean   => 26.5578231292517 |
| Median => 46.76537180436    | Median => 27               |
| 3rd-Qu => 64.085879880048   | 3rd-Qu => 42               |
| Max    => 96.994845223857   | Max    => 57               |
+-----------------------------+----------------------------+


Mean point of graph's vertexes:

In [37]:
my @mean-point = |$g2.vertex-coordinates.values.Array.&transpose.map({ $_.sum/$_.elems });

[46.00342655114355 26.5578231292517]

In [40]:
my @gs = 
[&euclidean-distance, &manhattan-distance, &chessboard-distance]
.map(-> &func { $g2.subgraph($g2.vertex-coordinates.grep({ &func($_.value, @mean-point) ≤ 18 })».key) })

[Graph(vertexes => 179, edges => 230, directed => False) Graph(vertexes => 116, edges => 149, directed => False) Graph(vertexes => 223, edges => 286, directed => False)]

DOT-plot (highlight vertexes with degree ≤ 2):

In [41]:
#%html
@gs.map( -> $g { $g.dot( 
    highlight => {IndianRed => $g.vertex-degree(:p).grep(*.value≤2)».key},
    :!node-labels, 
    node-width => 1.2,
    node-height => 1.2,
    node-fill-color => '#FFC000', 
    node-shape => <circle hexagon>.pick,
    graph-size => (4, 4),
    edge-color => 'Tan',
    edge-width => 28,
    engine => 'neato', 
):svg }).join("\n\n")

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 12.1.2 (20240928.0832)
 -->
<!-- Pages: 1 -->
 
 
 
<!-- 532 -->
 
 532 
 
 
<!-- 553 -->
 
 553 
 
 
<!-- 532--553 -->
 
 532--553 
 
 
<!-- 533 -->
 
 533 
 
 
<!-- 532--533 -->
 
 532--533 
 
 
<!-- 620 -->
 
 620 
 
 
<!-- 641 -->
 
 641 
 
 
<!-- 620--641 -->
 
 620--641 
 
 
<!-- 621 -->
 
 621 
 
 
<!-- 620--621 -->
 
 620--621 
 
 
<!-- 493 -->
 
 493 
 
 
<!-- 514 -->
 
 514 
 
 
<!-- 493--514 -->
 
 493--514 
 
 
<!-- 536 -->
 
 536 
 
 
<!-- 537 -->
 
 537 
 
 
<!-- 536--537 -->
 
 536--537 
 
 
<!-- 557 -->
 
 557 
 
 
<!-- 536--557 -->
 
 536--557 
 
 
<!-- 361 -->
 
 361 
 
 
<!-- 382 -->
 
 382 
 
 
<!-- 361--382 -->
 
 361--382 
 
 
<!-- 328 -->
 
 328 
 
 
<!-- 349 -->
 
 349 
 
 
<!-- 328--349 -->
 
 328--349 
 
 
<!-- 329 -->
 
 329 
 
 
<!-- 328--329 -->
 
 328--329 
 
 
<!-- 388 -->
 
 388 
 
 
<!-- 409 -->
 
 409 
 
 
<!-- 388--409 -->
 
 388--409 
 
 
<!-- 389 -->
 
 389 
 
 
<!-- 388--389 -->
 
 388--389 
 
 
<!-- 281 -->
 
 281 
 
 
<!-- 302 -->
 
 302 
 
 
<!-- 281--302 -->
 
 281--302 
 
 
<!-- 574 -->
 
 574 
 
 
<!-- 575 -->
 
 575 
 
 
<!-- 574--575 -->
 
 574--575 
 
 
<!-- 595 -->
 
 595 
 
 
<!-- 574--595 -->
 
 574--595 
 
 
<!-- 513 -->
 
 513 
 
 
<!-- 534 -->
 
 534 
 
 
<!-- 513--534 -->
 
 513--534 
 
 
<!-- 404 -->
 
 404 
 
 
<!-- 425 -->
 
 425 
 
 
<!-- 404--425 -->
 
 404--425 
 
 
<!-- 405 -->
 
 405 
 
 
<!-- 404--405 -->
 
 404--405 
 
 
<!-- 558 -->
 
 558 
 
 
<!-- 579 -->
 
 579 
 
 
<!-- 558--579 -->
 
 558--579 
 
 
<!-- 559 -->
 
 559 
 
 
<!-- 558--559 -->
 
 558--559 
 
 
<!-- 323 -->
 
 323 
 
 
<!-- 344 -->
 
 344 
 
 
<!-- 323--344 -->
 
 323--344 
 
 
<!-- 278 -->
 
 278 
 
 
<!-- 299 -->
 
 299 
 
 
<!-- 278--299 -->
 
 278--299 
 
 
<!-- 279 -->
 
 279 
 
 
<!-- 278--279 -->
 
 278--279 
 
 
<!-- 430 -->
 
 430 
 
 
<!-- 431 -->
 
 431 
 
 
<!-- 430--431 -->
 
 430--431 
 
 
<!-- 451 -->
 
 451 
 
 
<!-- 430--451 -->
 
 430--451 
 
 
<!-- 553--574 -->
 
 553--574 
 
 
<!-- 473 -->
 
 473 
 
 
<!-- 494 -->
 
 494 
 
 
<!-- 473--494 -->
 
 473--494 
 
 
<!-- 305 -->
 
 305 
 
 
<!-- 326 -->
 
 326 
 
 
<!-- 305--326 -->
 
 305--326 
 
 
<!-- 403 -->
 
 403 
 
 
<!-- 382--403 -->
 
 382--403 
 
 
<!-- 383 -->
 
 383 
 
 
<!-- 382--383 -->
 
 382--383 
 
 
<!-- 600 -->
 
 600 
 
 
<!-- 579--600 -->
 
 579--600 
 
 
<!-- 619 -->
 
 619 
 
 
<!-- 640 -->
 
 640 
 
 
<!-- 619--640 -->
 
 619--640 
 
 
<!-- 342 -->
 
 342 
 
 
<!-- 343 -->
 
 343 
 
 
<!-- 342--343 -->
 
 342--343 
 
 
<!-- 363 -->
 
 363 
 
 
<!-- 342--363 -->
 
 342--363 
 
 
<!-- 448 -->
 
 448 
 
 
<!-- 449 -->
 
 449 
 
 
<!-- 448--449 -->
 
 448--449 
 
 
<!-- 469 -->
 
 469 
 
 
<!-- 448--469 -->
 
 448--469 
 
 
<!-- 539 -->
 
 539 
 
 
<!-- 409--430 -->
 
 409--430 
 
 
<!-- 327 -->
 
 327 
 
 
<!-- 348 -->
 
 348 
 
 
<!-- 327--348 -->
 
 327--348 
 
 
<!-- 285 -->
 
 285 
 
 
<!-- 306 -->
 
 306 
 
 
<!-- 285--306 -->
 
 285--306 
 
 
<!-- 346 -->
 
 346 
 
 
<!-- 367 -->
 
 367 
 
 
<!-- 346--367 -->
 
 346--367 
 
 
<!-- 347 -->
 
 347 
 
 
<!-- 346--347 -->
 
 346--347 
 
 
<!-- 551 -->
 
 551 
 
 
<!-- 572 -->
 
 572 
 
 
<!-- 551--572 -->
 
 551--572 
 
 
<!-- 529 -->
 
 529 
 
 
<!-- 510 -->
 
 510 
 
 
<!-- 511 -->
 
 511 
 
 
<!-- 510--511 -->
 
 510--511 
 
 
<!-- 260 -->
 
 260 
 
 
<!-- 260--281 -->
 
 260--281 
 
 
<!-- 261 -->
 
 261 
 
 
<!-- 260--261 -->
 
 260--261 
 
 
<!-- 368 -->
 
 368 
 
 
<!-- 369 -->
 
 369 
 
 
<!-- 368--369 -->
 
 368--369 
 
 
<!-- 368--389 -->
 
 368--389 
 
 
<!-- 573 -->
 
 573 
 
 
<!-- 594 -->
 
 594 
 
 
<!-- 573--594 -->
 
 573--594 
 
 
<!-- 371 -->
 
 371 
 
 
<!-- 601 -->
 
 601 
 
 
<!-- 600--601 -->
 
 600--601 
 
 
<!-- 600--621 -->
 
 600--621 
 
 
<!-- 426 -->
 
 426 
 
 
<!-- 447 -->
 
 447 
 
 
<!-- 426--447 -->
 
 426--447 
 
 
<!-- 427 -->
 
 427 
 
